In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import os
import timm

In [2]:
model = timm.create_model('efficientnet_b7', pretrained=True)
num_classes = 9
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=num_classes)


No pretrained weights exist or were found for this model. Using random initialization.


In [3]:
train_path = './train'
test_path = './test'

resize_trans = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

resize_train = torchvision.datasets.ImageFolder(root=train_path, transform=resize_trans)
resize_test = torchvision.datasets.ImageFolder(root=test_path, transform=resize_trans)

train_loader = DataLoader(resize_train, batch_size=32, shuffle=True)
test_loader = DataLoader(resize_test, batch_size=32, shuffle=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 10 == 9:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}")
            running_loss = 0.0


[1, 10] loss: 2.563
[1, 20] loss: 2.356
[1, 30] loss: 2.177
[1, 40] loss: 1.939
[1, 50] loss: 1.755
[1, 60] loss: 1.469
[1, 70] loss: 1.452
[1, 80] loss: 1.436
[1, 90] loss: 1.285
[1, 100] loss: 1.283
[1, 110] loss: 1.330
[1, 120] loss: 1.264
[1, 130] loss: 1.160
[1, 140] loss: 0.993
[1, 150] loss: 0.994
[1, 160] loss: 1.065
[1, 170] loss: 1.083
[1, 180] loss: 0.967
[1, 190] loss: 0.956
[1, 200] loss: 0.857
[1, 210] loss: 0.885
[1, 220] loss: 0.919
[1, 230] loss: 0.839
[1, 240] loss: 0.824
[1, 250] loss: 0.777
[1, 260] loss: 0.672


In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

Accuracy: 72.67631947687997%
